# Now, we can start a new training job

We'll send a zip file called **trainingjob.zip**, with the following structure:
 - trainingjob.json (Sagemaker training job descriptor)
 - baseline.json (Sagemaker monitoring baseline parameters)
 - assets/deploy-model-prd.yml (Cloudformation for deploying our model into Production)
 - assets/deploy-model-dev.yml (Cloudformation for deploying our model into Development)

## Let's start by defining the hyperparameters for the aglroithm

In [ ]:
hyperparameters = {
    "epochs": 100,
    "batch_size": 128,
}

## Then, let's  create the trainingjob descriptor

In [ ]:
import time
import sagemaker
import boto3

sts_client = boto3.client("sts")

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

prefix='iris-model'
account_id = sts_client.get_caller_identity()["Account"]
region = boto3.session.Session().region_name
training_image = '{}.dkr.ecr.{}.amazonaws.com/{}:latest'.format(account_id, region, prefix)
roleArn = "arn:aws:iam::{}:role/MLOps".format(account_id)
timestamp = time.strftime('-%Y-%m-%d-%H-%M-%S', time.gmtime())
job_name = prefix + timestamp

training_params = {}

# Here we set the reference for the Image Classification Docker image, stored on ECR (https://aws.amazon.com/pt/ecr/)
training_params["AlgorithmSpecification"] = {
    "TrainingImage": training_image,
    "TrainingInputMode": "File"
}

# The IAM role with all the permissions given to Sagemaker
training_params["RoleArn"] = roleArn

# Here Sagemaker will store the final trained model
training_params["OutputDataConfig"] = {
    "S3OutputPath": 's3://{}/{}'.format(bucket, prefix)
}

# This is the config of the instance that will execute the training
training_params["ResourceConfig"] = {
    "InstanceCount": 1,
    "InstanceType": "ml.m4.xlarge",
    "VolumeSizeInGB": 30
}

# The job name. You'll see this name in the Jobs section of the Sagemaker's console
training_params["TrainingJobName"] = job_name

for i in hyperparameters:
    hyperparameters[i] = str(hyperparameters[i])
    
# Here you will configure the hyperparameters used for training your model.
training_params["HyperParameters"] = hyperparameters

# Training timeout
training_params["StoppingCondition"] = {
    "MaxRuntimeInSeconds": 360000
}

# The algorithm currently only supports fullyreplicated model (where data is copied onto each machine)
training_params["InputDataConfig"] = [{
    "ChannelName": "training",
    "DataSource": {
        "S3DataSource": {
            "S3DataType": "S3Prefix",
            "S3Uri": 's3://{}/{}/input/training'.format(bucket, prefix),
            "S3DataDistributionType": "FullyReplicated"
        }
    },
    "ContentType": "text/csv",
    "CompressionType": "None"
},{
    "ChannelName": "validation",
    "DataSource": {
        "S3DataSource": {
            "S3DataType": "S3Prefix",
            "S3Uri": 's3://{}/{}/input/validation'.format(bucket, prefix),
            "S3DataDistributionType": "FullyReplicated"
        }
    },
    "ContentType": "text/csv",
    "CompressionType": "None"
}]
training_params["Tags"] = []

## Before we start the training process, we need to upload our dataset to S3

In [ ]:
!head -2 input/data/training/train.csv

In [ ]:
!head -2 input/data/validation/test.csv

In [ ]:
train_loc = sagemaker_session.upload_data(path='input/data/training', key_prefix='iris-model/input/training')
val_loc = sagemaker_session.upload_data(path='input/data/validation', key_prefix='iris-model/input/validation')

print('training: {}\nvalidation: {}'.format(train_loc, val_loc))

## Create a baselining job from the validation output

Use the output predictions from testing for baseline file.  Make sure we have headers on this file

In [ ]:
# Inspect the output predictions (NOTE: if using scientific format these will be treated as strings)
baseline_file = 'output/data/predictions.csv'
!head -2 $baseline_file

In [ ]:
# Upload the predictions as baseline file
boto3.Session().resource('s3').Bucket(bucket).Object(baseline_file).upload_file(baseline_file)

In [ ]:
# copy over the training dataset to Amazon S3 (if you already have it in Amazon S3, you could reuse it)
baseline_prefix = prefix + '/baselining'
baseline_results_prefix = baseline_prefix + '/results'

baseline_data_uri = 's3://{}/{}'.format(bucket,baseline_file)
baseline_results_uri = 's3://{}/{}'.format(bucket, baseline_results_prefix)
print('Baseline data file: {}'.format(baseline_data_uri))
print('Baseline results uri: {}'.format(baseline_results_uri))

Now that we have the training data ready in S3, let's kick off a job to `suggest` constraints. `DefaultModelMonitor.suggest_baseline(..)` kicks off a `ProcessingJob` using a SageMaker provided Model Monitor container to generate the constraints. Please edit the configurations to fit your needs.

In [ ]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat
from sagemaker import get_execution_role

role = get_execution_role()

my_default_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

my_default_monitor.suggest_baseline(
    baseline_dataset=baseline_data_uri,
    dataset_format=DatasetFormat.csv(header=True),
    output_s3_uri=baseline_results_uri,
    wait=True
)

In [ ]:
baseline_job = my_default_monitor.latest_baselining_job
status = baseline_job.describe()['ProcessingJobStatus']
if status != 'Completed':
    raise(Exception('Processing job not completed, status: {}'.format(status)))
    
baseline_result_uri  = baseline_job.outputs[0].destination

### Explore the generated constraints and statistics

In [ ]:
import pandas as pd

schema_df = pd.io.json.json_normalize(baseline_job.baseline_statistics().body_dict["features"])
schema_df.head(10)

In [ ]:
constraints_df = pd.io.json.json_normalize(baseline_job.suggested_constraints().body_dict["features"])
constraints_df.head(10)

Save data capture config for the baseline, as well as monitoring resports and baseline constaints and statistics

In [ ]:
data_capture_uri = 's3://{}/{}/datacapture'.format(bucket, prefix)
monitoring_reports_uri = 's3://{}/{}/monitoring/reports'.format(bucket, prefix)

print('data capture uri: {}'.format(data_capture_uri))
print('monitoring reports: {}'.format(monitoring_reports_uri))

In [ ]:
monitoring_params = {
    'DataCaptureUri': data_capture_uri,
    'BaselineStatisticsUri':  baseline_result_uri + '/constraints.json',
    'BaselineConstraintsUri': baseline_result_uri + '/constraints.json',
    'ScheduleReportsUri': monitoring_reports_uri,
    'ScheduleRoleArn': roleArn
}

## Alright! Now it's time to start the training process

Create a cloudformation template package which includes inputs from trainingjob.json including Sha256 hash for forcing a Lambda redployment [see PR](https://github.com/awslabs/serverless-application-model/pull/1376)

In [ ]:
import json

# TEMP: Write a new file to the API directory to force refresh
with open('../../api/trainingjob.json', 'w') as f:
    json.dump(training_params, f)
with open('../../api/monitoring.json', 'w') as f:
    json.dump(monitoring_params, f)

!ls ../../api

Install the [custom resource helper](https://github.com/aws-cloudformation/custom-resource-helper) into the cfn folder

In [ ]:
!pip install -t ../../cfn crhelper

Generate the cloud formation template with API serverless endpoints uploading code to sagemaker bucket

In [ ]:
!aws cloudformation package --template-file ../../assets/deploy-model-prd.yml \
    --output-template-file ../../assets/template-model-prd.yml --s3-bucket $bucket

Verify the template has been generated correctly

In [ ]:
!cat ../../assets/template-model-prd.yml

In [ ]:
import boto3
import io
import zipfile
import json

s3 = boto3.client('s3')
sts_client = boto3.client("sts")

session = boto3.session.Session()

account_id = sts_client.get_caller_identity()["Account"]
region = session.region_name

bucket_name = "mlops-%s-%s" % (region, account_id)
key_name = "training_jobs/iris_model/trainingjob.zip"

zip_buffer = io.BytesIO()
with zipfile.ZipFile(zip_buffer, 'a') as zf:
    zf.writestr('trainingjob.json', json.dumps(training_params))
    zf.writestr('monitoring.json', json.dumps(monitoring_params)) 
    zf.writestr('assets/deploy-model-prd.yml', open('../../assets/template-model-prd.yml', 'r').read())
    zf.writestr('assets/deploy-model-dev.yml', open('../../assets/deploy-model-dev.yml', 'r').read())

zip_buffer.seek(0)

s3.put_object(Bucket=bucket_name, Key=key_name, Body=bytearray(zip_buffer.read()))

### Ok, now open the AWS console in another tab and go to the CodePipeline console to see the status of our building pipeline

> Finally, click here [NOTEBOOK](04_Check%20Progress%20and%20Test%20the%20endpoint.ipynb) to see the progress and test your endpoint